In [1]:
import os

import pandas as pd
import igraph as ig
import numpy as np
import leidenalg as la

from scipy.stats import zscore

import matplotlib.pyplot as plt
import seaborn as sns

# Functions

In [2]:
def genomic_position_stackbar (communities : list  , annotation_df : pd.DataFrame) : 
    final_annotation_df = pd.DataFrame()
    for num , community in enumerate(communities) :
        community_anontation = annotation_df.loc[list(communities)[num],:]
        community_anontation["community_numebr"] = "community_" + str(num)
        final_annotation_df = pd.concat([final_annotation_df  , community_anontation] , axis = 0 )

    return final_annotation_df


def transform_annotation_homer (path_file : str , filter_up = 0 , filter_down = 0) :
    annotation_gene = pd.read_csv(path_file,
                         delimiter="\t")
                         
    if filter_up != 0 :
        annotation_gene = annotation_gene.loc[(annotation_gene["Distance to TSS"] <= filter_up) & (annotation_gene["Distance to TSS"] >= filter_down) , :]

    annotation_gene = annotation_gene[["Chr","Start","End" ,'Annotation','Gene Name']]
    annotation_gene[['GenomicRegion', '_']] = annotation_gene.Annotation.str.split(" \\(", expand = True)
    annotation_gene['Start'] =  annotation_gene['Start'] -1
    annotation_gene["Sites"] = annotation_gene['Chr'].astype(str) +"-"+ annotation_gene["Start"].astype(str)+"-"+ annotation_gene["End"].astype(str)
    annotation_gene.drop( ['Chr','Start','End' , 'Annotation' , '_'] , axis = 1 , inplace=True)
    annotation_gene.set_index("Sites" , inplace=True)
    return annotation_gene

# check if there is any overlap between communities
# Function to check overlaps between two lists of lists
def count_overlaps(list1, list2):
    overlaps = []
    
    # Iterate over each pair of sublists
    for sublist1 in list1:
        overlap_row = []
        for sublist2 in list2:
            # Find the common elements using set intersection
            common_elements = set(sublist1).intersection(sublist2)
            overlap_row.append(len(common_elements))  # Count of common elements
        overlaps.append(overlap_row)
    
    return overlaps

def overlap_ratios(list1, list2):
    ratios = []
    
    # Iterate over each pair of sublists
    for sublist1 in list1:
        ratio_row = []
        for sublist2 in list2:
            # Find the common elements using set intersection
            common_elements = set(sublist1).intersection(sublist2)
            
            # Calculate the ratio of common elements to the size of sublist2
            if len(sublist2) > 0:
                ratio = len(common_elements) / len(sublist2)
            else:
                ratio = 0  # Handle edge case where sublist2 is empty (avoid division by zero)
            
            ratio_row.append(ratio)
        ratios.append(ratio_row)
    
    return ratios


def loading_comminity_from_folder (path_file : str) :
    onlyfiles = [f for f in os.listdir(path_file)]
    communities = pd.DataFrame()
    for f in onlyfiles :
        df = pd.read_csv(path_file+f ,delimiter="\t" , names = ['chr','start','end'])
        df["sites"] = df['chr'].astype(str) +"-"+ df["start"].astype(str)+"-"+ df["end"].astype(str)
        
        df.drop(['chr','start','end'], axis  = 1 , inplace = True)
        df['community'] = f.split(".tsv")[0]
        communities = pd.concat([communities , df])
    return communities 

# THP1 analysis

In [5]:
# Loading correlation matrix
e = pd.read_hdf('/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/CellReport/out_mutlicov_matrix/lognorm_edgeR_limma_countsInCellReport_TheiCounts_NoStatic_mean_Spearman.h5')
e_df = e.reset_index()

# Freeing space
del e

In [6]:
# Loading correlation matrix into a ig.Graph
G = ig.Graph.TupleList(e_df.itertuples(index=False), directed=False, weights=None, edge_attrs = 'corr')

In [7]:
# Evaluating network characteristics
print(G.vcount())
print(G.ecount())
print(G.density())

43446
80873018
0.08569274336648137


## Evaluating Best resolution parameters

In [ ]:
for x in np.linspace(0.1 , 2.0 , 21) :
    communities = la.find_partition( G , la.RBConfigurationVertexPartition  , resolution_parameter = x , weights = 'corr' , seed = 1234 ) #la.RBConfigurationVertexPartition la.CPMVertexPartition
    if x == 0.1 :
        data = {
            "Node": range(len(G.vs)),
            "Cluster_0.1": communities.membership
        }
        cluster_table = pd.DataFrame(data)

    else :
        cluster_table[ "Cluster_" + str(x)] = communities.membership


In [ ]:
cluster_table.to_csv("/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/CellReport/communities_k27/res_1_5/cluster_tree_all_res.tsv" , sep = "\t")

In [ ]:
modularity_score = []
for x in list(np.arange(0.1 , 2.0 , 21)) :
    communities = la.find_partition( G , la.RBConfigurationVertexPartition  , resolution_parameter = x , weights = 'corr' , seed = 1234 ) #la.RBConfigurationVertexPartition la.CPMVertexPartition
    modularity = G.modularity(communities.membership)
    modularity_score.append(modularity)

In [ ]:
plt.plot(np.arange(0.1 , 2.0 , 21) , modularity_score)

## Community Detection

In [8]:
communities = la.find_partition( G , la.RBConfigurationVertexPartition  , resolution_parameter = 1.5 , weights = 'corr' , seed = 1234 )

In [9]:
for x in communities :
    if len(x) > 100 :
        print(len(x))

8814
8315
7341
6689
4872
4075
3340


In [ ]:
values =[ len(x) for x in communities if len(x)>100]

sns.barplot(x=values , y=list(range(len(values))) , orient = "h" , color = 'black' )

# Setting Lables
plt.xlabel("Communities Size")
plt.ylabel("Communities")

#plt.savefig("/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/CellReport/pictures/communities_size.png", format="png", dpi=300, bbox_inches="tight")

# Show plot
plt.show()

In [ ]:
## Communities Spearman ROwnames
communities_list_sp = []
for n , community in enumerate(communities) :
    if len(community) > 100 :
        community_list = [G.vs[node_pos]["name"] for node_pos in community]
        communities_list_sp.append(community_list)

## Overlaps with known communities

In [ ]:
sites_th1_cellreport = pd.read_csv("/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/CellReport/their_regions_timepoints.bed" , sep = "\t" , header = None , names = ['chromosome' , 'start','end','annotation'])
data_matrix_df = pd.read_csv("/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/CellReport/out_mutlicov_matrix/lognorm_edgeR_limma_countsInCellReport_TheiCounts_NoStatic_mean.tsv" , sep = "\t" , index_col = "peaks" )


sites_th1_cellreport["sites"] = sites_th1_cellreport['chromosome'].astype(str) +"-"+ sites_th1_cellreport["start"].astype(str)+"-"+ sites_th1_cellreport["end"].astype(str)
sites_th1_cellreport = sites_th1_cellreport[['sites' , 'annotation']]
sites_th1_cellreport = sites_th1_cellreport.drop_duplicates('sites')
mask = sites_th1_cellreport['sites'].isin(data_matrix_df.index)
sites_th1_cellreport = sites_th1_cellreport.loc[mask ,]

In [ ]:
communities_list_comparison =[]
communities_name = []
for x in ['dn.early','dn.grad','dn.mid','dn.var','up.grad','up.late','up.mid','up.var'] : #list(set(sites_th1_cellreport['annotation']))
    df_sel = sites_th1_cellreport.loc[ sites_th1_cellreport['annotation'] == x , :]
    communities_list_comparison.append(list(df_sel["sites"]))
    communities_name.append(x)

In [ ]:
overlap_counts_sp_control = overlap_ratios(communities_list_comparison , communities_list_sp )#communities_list_to_plot_annotations
overlap_counts_sp_control = np.transpose(overlap_counts_sp_control)
#plt.imshow(overlap_counts_sp_control, interpolation='nearest') 
#plt.figure(figsize = (9,9))
sns.heatmap(np.round(overlap_counts_sp_control,2), annot=True, cmap="viridis", cbar=True ,  xticklabels=communities_name )

# Add colorbar 
#plt.colorbar() 
  
# plt.title("Heatmap with color bar") 
# plt.savefig("/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/CellReport/pictures/Overlap_communities_and_mine.png", bbox_inches='tight' , dpi=300)
plt.show() 

## Plotting Communities Trends

In [ ]:
for x in range(0,7) :
    data_matrix_df_k27 = pd.read_csv("/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/CellReport/out_mutlicov_matrix/lognorm_edgeR_limma_countsInCellReport_TheiCounts_mean.tsv" , sep = "\t" , index_col = "peaks" )
    count_df = data_matrix_df_k27.loc[communities_list_sp[x],:].apply(zscore , axis=1)
    count_df = count_df.stack().reset_index()
    count_df.columns = ['peaks','variable', 'value']
    count_df['community'] = str(x)
    
    if x == 0 :
        final_df = count_df
    else :
        final_df = pd.concat([final_df , count_df])

plt.figure(figsize=(20, 12))

# Create the violin plot
#sns.violinplot(data=final_df,x="variable", y="value", hue="community", scale='width', inner=None)

# Facet by Cluster
g = sns.FacetGrid(data=final_df,row='community', sharey=False, height=3, aspect=2, despine=False) 
g.map_dataframe(sns.violinplot,x="variable", y="value", hue="community", scale='width', fill=False , color = 'gray')
g.map_dataframe(sns.lineplot,  x = 'variable' , y = 'value' , markers=False , color='black' , lw=2)
g.figure.subplots_adjust(wspace=0, hspace=0.1)
g.set_axis_labels("", "Accessibilty Z-score") 
g.set_titles(row_template="")

# Customize aesthetics
for ax in g.axes.flat:
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')
    ax.set_xlabel("variable")
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_color('black')
    ax.spines['left'].set_color('black')

g.despine(left=True)
# plt.savefig('/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/CellReport/pictures/stacked_trends.png', bbox_inches='tight' , dpi=300)
plt.show()

In [ ]:
# Plotting Genomic Annotations
annotation = transform_annotation_homer("/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/CellReport/consensus_peaks_TH_cellreport_annotations.txt")
final_annotation_prova = genomic_position_stackbar(communities_list_sp, annotation) #_noPC
plt.figure(figsize = (10 , 10 ))
#fig , ax = plt.subplots()
#fig.set_size_inches(18.5, 10.5)
final_annotation_prova.groupby('community_numebr')['GenomicRegion'].value_counts(normalize=True).unstack('GenomicRegion').plot.bar(stacked=True ) 
plt.legend(loc = 'upper right' , bbox_to_anchor=(1.35, 0.75))

#plt.savefig('/mnt/nas-safu/analysis/PhDsdigiove/Vienna/Pictures/stacked_community.png', bbox_inches='tight' , dpi=300)

In [ ]:
# Save Community to files
for comm_number,commnity in enumerate(communities_list_sp) :
    if len(commnity) > 100:
        save_path = "/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/CellReport/communities_res_08_bothrep/community" + str(comm_number) + ".bed" 
        with open(save_path, 'w') as f:
            for line in commnity:
                line = line.split('-')
                line = '\t'.join(line)
                f.write(f"{line}\n")

## Plot relations ATAC and RNA

In [ ]:
data_matrix_df_genes = pd.read_csv("/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/CellReport/out_mutlicov_matrix/Genes_lognorm_edgeR_limma_countsInCellReport_TheiCounts_All_mean.tsv" , sep = "\t" , index_col = "hgnc" )
data_matrix_df_genes = data_matrix_df_genes.loc[communities_list_sp[2],:].reset_index().drop(columns = 'hgnc')
data_matrix_df_genes['Dataset'] = 'RNAseq'

data_matrix_df_k27 = pd.read_csv("/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/CellReport/out_mutlicov_matrix/lognorm_edgeR_limma_countsInCellReport_TheiCounts_NoStatic_mean.tsv" , sep = "\t" , index_col = "peaks" )
data_matrix_df_k27 = data_matrix_df_k27.loc[communities_list_sp_h3k27.loc[communities_list_sp_h3k27['community'] == 'community0.bed', 'sites'],:].reset_index().drop(columns = 'peaks')
data_matrix_df_k27['Dataset'] = 'H3K27'

final_df = pd.concat([data_matrix_df_genes ,data_matrix_df_k27] , axis = 0)

In [ ]:
comm0 = transform_annotation_homer("/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/CellReport/communities_k27/res_1_5/annotated_feature/community1.txt")
target_regions = list(set(list(comm0.index)))
target_genes = list(set(list(comm0["Gene Name"])))
data_matrix_df_genes = pd.read_csv("/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/CellReport/out_mutlicov_matrix/Genes_lognorm_edgeR_limma_countsInCellReport_TheiCounts_All_mean.tsv" , sep = "\t" , index_col = "hgnc" )
target_genes_trans = [x for x in target_genes if x in list(data_matrix_df_genes.index)]
data_matrix_df_genes = data_matrix_df_genes.loc[target_genes_trans,:].reset_index().drop(columns = 'hgnc')
data_matrix_df_genes['Dataset'] = 'RNAseq'
data_matrix_df_k27 = pd.read_csv("/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/CellReport/out_mutlicov_matrix/lognorm_edgeR_limma_countsInCellReport_TheiCounts_NoStatic_mean.tsv" , sep = "\t" , index_col = "peaks" )
data_matrix_df_k27 = data_matrix_df_k27.loc[target_regions,:].reset_index().drop(columns = 'peaks')
data_matrix_df_k27['Dataset'] = 'H3K27'

final_df = pd.concat([data_matrix_df_genes ,data_matrix_df_k27] , axis = 0)
#only_genes_in_comm3RNA_comm1ATAC
final_df2 = final_df.iloc[: , 0:-1].apply(zscore , axis=1)
final_df2['Dataset'] = final_df['Dataset']
sns.lineplot(pd.melt(final_df2, id_vars = ['Dataset']) , x = 'variable' , y = 'value'  , hue = 'Dataset', markers=True) 
plt.gca().set_xticklabels(['0min','30min','60min','90min','120min','240min','360min','1440min'])
print(data_matrix_df_genes.shape)

#plt.savefig('/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/CellReport/pictures/comm1_ATAC_RNA.png', bbox_inches='tight' , dpi=300)

# Liver Development

In [ ]:
# load correlation table
e = pd.read_hdf('/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/liver_devel_mouse_ENCODE/network_files/normalized_samplemean_multicov_all_sites_all_timepoint.h5')
e_df = e.reset_index()
del e

In [ ]:
G = ig.Graph.TupleList(e_df.itertuples(index=False), directed=False, weights=None, edge_attrs = 'corr')

In [ ]:
print(len(list(G.vs)))
print(len(list(G.es)))
print(G.density())

## Evaluating Best resolution parameters

In [ ]:
modularity_score = []
for x in list(np.arange(0.1, 2.0, 0.1)) :
    communities = la.find_partition( G , la.RBConfigurationVertexPartition  , resolution_parameter = x , weights = 'corr' , seed = 1234 ) #la.RBConfigurationVertexPartition la.CPMVertexPartition
    modularity = G.modularity(communities.membership)
    modularity_score.append(modularity)

plt.plot(np.arange(0.1, 2.0, 0.1) , modularity_score)

In [ ]:
for x in np.linspace(0.1 , 2.0 , 21) :
    communities = la.find_partition( G , la.RBConfigurationVertexPartition  , resolution_parameter = x , weights = 'corr' , seed = 1234 ) #la.RBConfigurationVertexPartition la.CPMVertexPartition
    if x == 0.1 :
        data = {
            "Node": range(len(G.vs)),
            "Cluster_0.1": communities.membership
        }
        cluster_table = pd.DataFrame(data)

    else :
        cluster_table[ "Cluster_" + str(x)] = communities.membership

# cluster_table.to_csv("/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/liver_devel_mouse_ENCODE/communities/cluster_tree_all_res.tsv" , sep = "\t")


## Community Detection

In [ ]:
communities = la.find_partition( G , la.RBConfigurationVertexPartition  , resolution_parameter = 0.8 , weights = 'corr' , seed = 1234 ) #la.RBConfigurationVertexPartition la.CPMVertexPartition\

In [ ]:
for x in communities :
    if len(x) > 100 :
        print(len(x))

In [ ]:
## Communities Spearman ROwnames
communities_list_sp = []
for n , community in enumerate(communities) :
    if len(community) > 100 :
        community_list = [G.vs[node_pos]["name"] for node_pos in community]
        communities_list_sp.append(community_list)

In [ ]:
# Example data
values =[ len(x) for x in communities_list_sp if len(x)>100]

# Plot
sns.barplot(x=values , y=list(range(len(values))) , orient = "h" , color = 'black' )

# Label the axes
plt.xlabel("Communities Size")
plt.ylabel("Communities")

# plt.savefig("/mnt/nas-safu/analysis/PhDsadigiove/method_coAcces/data/liver_devel_mouse_ENCODE/communities/res_0_8/communities_size_08_final.png", format="png", dpi=300, bbox_inches="tight")

# Show the plot
plt.show()

## Plotting Communities Trends

In [ ]:
for x in range(0,5) :
    data_matrix_df_k27 = pd.read_csv("/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/liver_devel_mouse_ENCODE/network_files/normalized_samplemean_multicov_all_sites_all_timepoint.tsv" , sep = "\t" , index_col = "peaks" )
    count_df = data_matrix_df_k27.loc[communities_list_sp[x],:].apply(zscore , axis=1)
    count_df = count_df.stack().reset_index()
    count_df.columns = ['peaks','variable', 'value']
    count_df['community'] = str(x)
    
    if x == 0 :
        final_df = count_df
    else :
        final_df = pd.concat([final_df , count_df])

plt.figure(figsize=(20, 12))

# Create the violin plot
#sns.violinplot(data=final_df,x="variable", y="value", hue="community", scale='width', inner=None)

# Facet by Cluster
g = sns.FacetGrid(data=final_df,row='community', sharey=False, height=3, aspect=2, despine=False) 
g.map_dataframe(sns.violinplot,x="variable", y="value", hue="community", scale='width', fill=False , color = 'gray')
g.map_dataframe(sns.lineplot,  x = 'variable' , y = 'value' , markers=False , color='black' , lw=2)
g.figure.subplots_adjust(wspace=0, hspace=0.1)
g.set_axis_labels("", "Accessibilty Z-score") 
g.set_titles(row_template="")

# Customize aesthetics
for ax in g.axes.flat:
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')
    ax.set_xlabel("variable")
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_color('black')
    ax.spines['left'].set_color('black')

g.despine(left=True)
plt.savefig('/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/liver_devel_mouse_ENCODE/pictures/violin_trends_08/stacked_trends.png', bbox_inches='tight' , dpi=300)

plt.show()

In [ ]:
files = os.listdir('/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/liver_devel_mouse_ENCODE/communities/res_0_8/homer_annotations/')
i = 0
for f in files:
    et = transform_annotation_homer('/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/liver_devel_mouse_ENCODE/communities/res_0_8/homer_annotations/{}'.format(f))
    et['community'] = f

    if i == 0 :
        final_df_annot = et
    else :
        final_df_annot = pd.concat([final_df_annot , et ])
    i = 1

final_annotation_prova = genomic_position_stackbar(communities_list_sp, final_df_annot) #_noPC

x_var, y_var = "community_numebr", "GenomicRegion"
df_grouped = final_annotation_prova.groupby(x_var)[y_var].value_counts(normalize=True).unstack(y_var)
df_grouped = df_grouped.iloc[::-1]
df_grouped.plot.barh(stacked=True)
plt.legend(
    bbox_to_anchor=(0.5, 1.02),
    loc="lower center",
    borderaxespad=0,
    frameon=False,
    ncol=3,
)
for ix, row in df_grouped.reset_index(drop=True).iterrows():
    cumulative = 0
    for element in row:
        if element == element and element > 0.1:
            plt.text(
                cumulative + element / 2,
                ix,
                f"{int(element * 100)} %",
                va="center",
                ha="center",
            )
        cumulative += element
plt.tight_layout()
# plt.savefig('/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/liver_devel_mouse_ENCODE/pictures/stacked_community_inverted_res_0_8.png', bbox_inches='tight' , dpi=300)

# B-ALL analysis

In [ ]:
# load correlation table
e = pd.read_hdf('/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/BALL/allPatientsPatientsNOMERGED_Ball_Multicov_nfcore_05.h5')
e_df = e.reset_index()
del e

In [ ]:
G = ig.Graph.TupleList(e_df.itertuples(index=False), directed=False, weights=None, edge_attrs = 'corr')

In [ ]:
print(len(list(G.vs)))
print(len(list(G.es)))
print(G.density())

## Evaluating Best resolution parameters

In [ ]:
modularity_score = []
community_number = []
for x in list(np.arange(0.1, 2.0, 0.1)) :
    communities = la.find_partition( G , la.RBConfigurationVertexPartition  , resolution_parameter = x , weights = 'corr' , seed = 1234 ) #la.RBConfigurationVertexPartition la.CPMVertexPartition
    modularity = G.modularity(communities.membership)
    modularity_score.append(modularity)
    community_number.append(len(list(set(communities.membership))))
    if x == 0.1 :
        data = {
            "Node": range(len(G.vs)),
            "Cluster_0.1": communities.membership
        }
        cluster_table = pd.DataFrame(data)

    else :
        cluster_table[ "Cluster_" + str(x)] = communities.membership

In [ ]:
cluster_table.to_csv("/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/liver_devel_mouse_ENCODE/communities/cluster_tree_all_res_th05_nfcore.tsv" , sep = "\t")

In [ ]:
plt.plot(np.arange(0.1, 2.0, 0.1) , modularity_score)

## Community detection

In [ ]:
communities = la.find_partition( G , la.RBConfigurationVertexPartition  , resolution_parameter = 0.9 , weights = 'corr' , seed = 1234 )

In [ ]:
## Communities Spearman ROwnames
communities_list_sp = []
for n , community in enumerate(communities) :
    if len(community) > 100 :
        community_list = [G.vs[node_pos]["name"] for node_pos in community]
        communities_list_sp.append(community_list)

In [ ]:
# Example data
values =[ len(x) for x in communities_list_sp if len(x)>100]

# Plot
sns.barplot(x=values , y=list(range(len(values))) , orient = "h" , color = 'black' )

# Label the axes
plt.xlabel("Communities Size")
plt.ylabel("Communities")

# plt.savefig("/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/BALL/communities_broad/res_0_9/community_size.png", format="png", dpi=300, bbox_inches="tight")

# Show the plot
plt.show()

## Plotting Community trends

In [1]:
%matplotlib inline

In [4]:
communities_list_sp_df = loading_comminity_from_folder("/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/BALL/communities_broad/res_0_9/beds/")

communities_list_sp = []
for comm in sorted(set(communities_list_sp_df.community)) :
    # print(comm)
    #print(len(communities_list_sp_df.loc[communities_list_sp_df['community' ] == comm , 'sites'].tolist()) ) 
    communities_list_sp.append(communities_list_sp_df.loc[communities_list_sp_df['community' ] == comm , 'sites'].tolist())

In [5]:
def boxplot_with_median(data, **kwargs):
    ax = plt.gca()
    #sns.boxplot(data=data,  x = 'original_column' , y = 'expression', ax=ax, order=None)
    
    # Compute median per facet and draw a line
    median = data["expression"].median()
    ax.axhline(median, color="red", linestyle="--", linewidth=2, label=f"Median: {median:.2f}")
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

In [ ]:
patient_groups = {"Healthy": 6, "Primary": 11 , "Remission":8 , "Relapse":8  }
order = ["Healthy", "Primary", "Remission", "Relapse"]
data_matrix_df = pd.read_csv("/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/BALL/allPatientsPatientsNOMERGED_Ball_Multicov_nfcore.tsv" , sep = "\t" , index_col = "peaks" )
data_matrix_df.columns = ["6_Healthy","9_Healthy","4_Healthy","7_Healthy","5_Healthy","8_Healthy","14_Primary","12_Primary","10_Primary","13_Primary","11_Primary","16_Primary","7_Primary","19_Primary","17_Primary","27_Primary","26_Primary","21_Remission","22_Remission","18_Remission","26_Remission","20_Remission","25_Remission","23_Remission","24_Remission","31_Relapse","28_Relapse","29_Relapse","33_Relapse","32_Relapse","27_Relapse","30_Relapse","14_Relapse",]

id_vars =list(data_matrix_df.index)
value_vars = [col for col in data_matrix_df.columns if col not in id_vars]

#myorder = ['MUD4', 'MUD5','MUD6', 'MUD7', 'MUD8', 'MUD9',  'ES26', 'ES27', 'ES10','ES11', 'ES12', 'ES13', 'ES16', 'ES7', 'ES17', "ES100",'ES19', 'ES14', 'REM23','REM24', 'REM25', 'REM26', 'REM20', 'REM21', 'REM18','REM18', 'REC29', 'REC30', 'REC31', 'REC32', 'REC33', 'REC14', 'REC27']

group_mapping = {}
start_idx = 0
for group_name, size in patient_groups.items():
    for i in range(size):
        group_mapping[value_vars[start_idx + i]] = group_name
    start_idx += size

data_matrix_df = pd.read_csv("/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/BALL/allPatientsPatientsNOMERGED_Ball_Multicov_nfcore.tsv" , sep = "\t" , index_col = "peaks" )
data_matrix_df.columns = ["6_Healthy","9_Healthy","4_Healthy","7_Healthy","5_Healthy","8_Healthy","14_Primary","12_Primary","10_Primary","13_Primary","11_Primary","16_Primary","7_Primary","19_Primary","17_Primary","27_Primary","26_Primary","21_Remission","22_Remission","18_Remission","26_Remission","20_Remission","25_Remission","23_Remission","24_Remission","31_Relapse","28_Relapse","29_Relapse","33_Relapse","32_Relapse","27_Relapse","30_Relapse","14_Relapse",]
data_matrix_df = data_matrix_df.apply(zscore , axis=1)

for x in range(0,len(communities_list_sp)) :
    #plt.figure(figsize=(3,5))
    melted = pd.melt(
        data_matrix_df.loc[communities_list_sp[x],:],
        value_vars=value_vars,
        var_name="original_column",
        value_name="expression"
    )
    melted["group"] = melted["original_column"].map(group_mapping)
    melted["group"] = pd.Categorical(melted["group"], categories= order, ordered=True)
    
    facet_colors = {
        "Healthy": "#66c5cc",
        "Primary": "#ff6767",
        "Remission": "#ffdb84",
        "Relapse":"#ff951f" 
    }

    print(x)
    g = sns.FacetGrid(melted, col="group" , sharex=False  ,ylim=(-5,5) )
    g.fig.set_size_inches(9, 5)
    g.map_dataframe(sns.violinplot, x = 'original_column' , y = 'expression' , hue = "group" , palette= facet_colors )
    g.map_dataframe(boxplot_with_median)
    g.set_titles("")
    g.set_axis_labels("", "Z-score")
    for ax in g.axes.flat:
        ax.tick_params(axis='both', labelsize=7)
    # plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
    #plt.gca().set_xticklabels(['0min','30min','60min','90min','120min','240min','360min','1440min']) 
    path_to_save = '/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/BALL/communities_broad/res_0_9/comm_trends/Facets_boxplots_normCountsOnly_comm' + str(x) + "_ATAC_trend.png"
    g.savefig(path_to_save, dpi=300, bbox_inches="tight")
    plt.show()

In [ ]:
files = os.listdir('/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/BALL/communities_broad/res_0_9/homer_annotation/')
i = 0
for f in files:
    et = transform_annotation_homer('/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/BALL/communities_broad/res_0_9/homer_annotation/{}'.format(f))
    et['community'] = f

    if i == 0 :
        final_df_annot = et
    else :
        final_df_annot = pd.concat([final_df_annot , et ])
    i = 1

final_annotation_prova = genomic_position_stackbar(communities_list_sp, final_df_annot) #_noPC

x_var, y_var = "community_numebr", "GenomicRegion"
df_grouped = final_annotation_prova.groupby(x_var)[y_var].value_counts(normalize=True).unstack(y_var)
df_grouped = df_grouped.iloc[::-1]
df_grouped.plot.barh(stacked=True)
plt.legend(
    bbox_to_anchor=(0.5, 1.02),
    loc="lower center",
    borderaxespad=0,
    frameon=False,
    ncol=3,
)
for ix, row in df_grouped.reset_index(drop=True).iterrows():
    cumulative = 0
    for element in row:
        if element == element and element > 0.1:
            plt.text(
                cumulative + element / 2,
                ix,
                f"{int(element * 100)} %",
                va="center",
                ha="center",
            )
        cumulative += element
plt.tight_layout()
# plt.savefig('/mnt/nas-safu/analysis/PhDsdigiove/method_coAcces/data/BALL/communities_broad/res_0_9/stacked_community_inverted.png', bbox_inches='tight' , dpi=300)